# `CLEMnet` predictions on RL012
---
#### Overview
* Model for typical results figure and whatnot

In [1]:
# device = cuda.get_current_device()
# device.reset()

In [2]:
import clemnet

In [3]:
from pathlib import Path
import random
from datetime import datetime
from numba import cuda 

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec

from skimage import io, exposure, color
from skimage import img_as_ubyte, img_as_float
from skimage.util import invert, crop
from skimage.transform import resize
from sklearn import preprocessing

from mpl_toolkits.axes_grid1 import make_axes_locatable
import cv2
from PIL import Image, ImageEnhance, ImageOps, ImageFilter

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

%matplotlib widget

In [4]:
# Set GPU
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

# Ignore annoying Futurewarning from `gray2rgba`
import warnings
warnings.simplefilter('ignore', FutureWarning)

# Set ... AUTOTUNE?
AUTOTUNE = tf.data.experimental.AUTOTUNE

# Indirectly enable autocomplete
%config Completer.use_jedi = False

### Create model
Need better resolution for insulin prediction

In [5]:
def get_model(input_shape=(1024, 1024), crop_=False, crop_width=None):
    """U-net-like convolutional neural network
    Parameters
    ----------
    input_shape : tuple
        Shape of input image data
    crop : bool
        Whether to include a cropping layer
    crop_width : int
        Number of pixels to crop from each border

    Returns
    -------
    model : `keras.Model`
        The model (duh)

    References
    ----------
    [1] U-Net: Convolutional Networks for Biomedical Image Segmentation
        https://arxiv.org/abs/1505.04597
    [2] Adapted from
        https://github.com/zhixuhao/unet/blob/master/model.py
    """
    # Create input layer
    input_shape = (*input_shape, 1) if len(input_shape) < 3 else input_shape
    inputs = layers.Input(shape=input_shape)

    # Set up keyword arguments for convolutional layers
    kwargs = {
        'activation': 'relu',
        'padding': 'same',
        'kernel_initializer': 'he_normal'
    }

    # Downsampling arm
    # ----------------
    # Block 1
    conv1 = layers.Conv2D(32, 3, **kwargs)(inputs)
    pool1 = layers.MaxPooling2D(2)(conv1)
    # Block 2
    conv2 = layers.Conv2D(64, 3, **kwargs)(pool1)
    pool2 = layers.MaxPooling2D(2)(conv2)
    # Block 3
    conv3 = layers.Conv2D(128, 3, **kwargs)(pool2)
    pool3 = layers.MaxPooling2D(2)(conv3)
    # Block 4
    conv4 = layers.Conv2D(256, 3, **kwargs)(pool3)
    pool4 = layers.MaxPooling2D(2)(conv4)
    # Block 5
    conv5 = layers.Conv2D(512, 3, **kwargs)(pool4)
    pool5 = layers.MaxPooling2D(2)(conv5)
    # Block 6
    conv6 = layers.Conv2D(1024, 3, **kwargs)(pool5)
    pool6 = layers.MaxPooling2D(2)(conv6)

    # Upsampling arm
    # --------------
    # Block 7
    conv7 = layers.Conv2D(1024, 3, **kwargs)(pool6)
    uppp7 = layers.UpSampling2D(2)(conv7)
    # Block 8
    merg8 = layers.concatenate([conv6, uppp7], axis=3)
    conv8 = layers.Conv2D(1024, 3, **kwargs)(merg8)
    uppp8 = layers.UpSampling2D(2)(conv8)
    # Block 9
    merg9 = layers.concatenate([conv5, uppp8], axis=3)
    conv9 = layers.Conv2D(512, 3, **kwargs)(merg9)
    uppp9 = layers.UpSampling2D(2)(conv9)
    # Block 10
    merg10 = layers.concatenate([conv4, uppp9], axis=3)
    conv10 = layers.Conv2D(256, 3, **kwargs)(merg10)
    conv10 = layers.Conv2D(2, 3, **kwargs)(conv10)

    # Additional upsampling
    uppp11 = layers.UpSampling2D(2)(conv10)

    # Output layer
    conv11 = layers.Conv2D(1, 1, activation='sigmoid')(uppp11)
    model = keras.Model(inputs=inputs, outputs=conv11)
    return model

In [6]:
shape_src = [1024, 1024]
# Set training parameters
repeat = True
n_repetitions = 4
shape_src = [1024, 1024]
shape_tgt = [256, 256]
augment = True
AUGMENTATIONS = {
    'flip': 1,
    'rotate': 1,
    'crop': 0.,
    'elastic': 0.3,
    'invert': 0.,
    'contrast': 0.3,
    'brightness': 0.3,
    'noise': 0.,
}
pad = False
padding = tf.constant([[16, 16],
                       [16, 16],
                       [ 0,  0]])
batch_size = 16
n_cores = 4
input_shape = (1024, 1024)

# Get model
crop_ = pad
crop_width = 16
model = get_model(input_shape=input_shape,
                  crop_=crop_, crop_width=crop_width)
# model.summary()
print('model OK')

2022-03-15 13:51:31.266760: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-03-15 13:51:32.797926: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:b1:00.0 name: Tesla P100-PCIE-12GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 11.91GiB deviceMemoryBandwidth: 511.41GiB/s
2022-03-15 13:51:32.798236: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-03-15 13:51:32.800135: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-03-15 13:51:32.801711: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2022-03-15 13:51:32.802031: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.

model OK


# Automated prediction Initializations
---


In [7]:
def mean_std_adjustments(m0, s0, m1, s1, image):
    std_factor = s1/s0
    mean_adjustment = m1-m0
    image = image+mean_adjustment
    image = tf.image.adjust_contrast(image[:,:,np.newaxis], std_factor)
    image = image.numpy()[:,:,0]
    return image

### Settings automated prediction analysis 

In [7]:
# Initialization 
# --------------------------------------------

zoom_level = 0
active_mask = 'OSTEM-1'
scale = 128

w_alpha = 0.80
w_beta = 1-w_alpha

fp_FASTEM_0 = '/home/rlane/FMML_DATA/20210224_FAST-EM_prototype'
fp_OSTEM_1 = '/home/rlane/FMML_DATA/20211206_RP_OTO/lil_EM_montaged'
fp_src = fp_OSTEM_1

# FASTEM-0
# [[[x0, x1], [y0, y1]]]
# Langerhans Island
# -----------------------------------
# fltr_manual = [[4, 14], [18, 25]]             # zoom_level = 0
# fltr_manual = [[4, 4], [18, 18]]             # zoom_level = 0
# fltr_manual = [[5, 6], [19, 20]]             # zoom_level = 0

#OSTEM-1
# fltr_manual = [[16, 16], [11, 11]]                # zoom_level = 0
fltr_manual = [[16, 20], [10, 15]]                # zoom_level = 0

sweep = True
preview_prediction = False
with_callbacks = False
save_file = True
threshold_mask = True

adjust_PIL = True
adjust_invert = True
adjust_sharpness = True
adjust_contrast = True
adjust_mean_std = True
adjust_clipping = False
    
# RL012 , FASTEM-0 and OSTEM-1: mu & sigma
std_RL012 = 0.287
mean_RL012 = 0.462
std_FASTEM_0 = 0.134
mean_FASTEM_0 = 0.487
std_OSTEM_1 = 0.134 
mean_OSTEM_1 = 0.644
std_BSE_1 = std_RL012 
mean_BSE_1 = mean_RL012
std_tgt_1 = std_OSTEM_1
mean_tgt_1 = mean_OSTEM_1

# Qualitatively obtained parameters, initialize
threshold_value = 0.0375
adjust_sharpness_level = 4.5
adjust_contrast_level = 1.2

threshold_start = 0
threshold_end = 0.09
sharpness_start = 0
sharpness_end = 6
contrast_start = 0
contrast_end = 3
range_length = 9

weights_list = [
'RL012_1024x1024_hoechst_zoomlevel_1_gaussian_blur_1_20220107_151123.h5',
'RL012_1024x1024_hoechst_zoomlevel_1_gaussian_blur_2_20220103_185956.h5',
'RL012_1024x1024_hoechst_zoomlevel_1_gaussian_blur_3_20211221_211240.h5',
'RL012_1024x1024_hoechst_zoomlevel_1_gaussian_blur_4_20211124_103409.h5',
'RL012_1024x1024_hoechst_zoomlevel_1_gaussian_blur_5_20220102_165203.h5',
'RL012_1024x1024_hoechst_zoomlevel_1_gaussian_blur_6_20220109_161845.h5'
]

# Prepare parameter sweep arrays
if sweep:
    threshold_value_array = np.linspace(threshold_start, 
                                        threshold_end, range_length)
    adjust_sharpness_level_array = np.linspace(sharpness_start, 
                                        sharpness_end, range_length)
    adjust_contrast_level_array = np.linspace(contrast_start, 
                                        contrast_end, range_length)
else:
    threshold_value_array = [threshold_value]
    adjust_sharpness_level_array = [adjust_sharpness_level]
    adjust_contrast_level_array = [adjust_contrast_level]

print('Threshold: \n' + str(threshold_value_array) 
      if sweep else 'Threshold:   ' + str(threshold_value_array[0]))
print('Sharpness: \n' + str(adjust_sharpness_level_array) 
      if sweep else 'Sharpness:   ' + str(adjust_sharpness_level_array[0]))
print('Contrast: \n' + str(adjust_contrast_level_array) 
      if sweep else 'Contrast:    ' + str(adjust_contrast_level_array[0]))

print()
print('Weights: (Gaussian Blur)')
for w in range(len(weights_list)):
    print(weights_list[w])

Threshold: 
[0.      0.01125 0.0225  0.03375 0.045   0.05625 0.0675  0.07875 0.09   ]
Sharpness: 
[0.   0.75 1.5  2.25 3.   3.75 4.5  5.25 6.  ]
Contrast: 
[0.    0.375 0.75  1.125 1.5   1.875 2.25  2.625 3.   ]

Weights: (Gaussian Blur)
RL012_1024x1024_hoechst_zoomlevel_1_gaussian_blur_1_20220107_151123.h5
RL012_1024x1024_hoechst_zoomlevel_1_gaussian_blur_2_20220103_185956.h5
RL012_1024x1024_hoechst_zoomlevel_1_gaussian_blur_3_20211221_211240.h5
RL012_1024x1024_hoechst_zoomlevel_1_gaussian_blur_4_20211124_103409.h5
RL012_1024x1024_hoechst_zoomlevel_1_gaussian_blur_5_20220102_165203.h5
RL012_1024x1024_hoechst_zoomlevel_1_gaussian_blur_6_20220109_161845.h5


In [8]:
fps_src = list(Path(fp_src).glob('*/*_*_*.png'))

# Get DataFrame for predicting
df_tst = clemnet.get_DataFrame(fps_src)
df_tst = df_tst.loc[df_tst['zoom'] == zoom_level]
# Filter out a test section
df_tst = df_tst.loc[(df_tst['z'] != 1)]
df_tst_filtered = pd.DataFrame()
# # Must be done per section, first x, than y
fltr  = pd.DataFrame()

for i, (z, grp) in enumerate(df_tst.groupby('z')):
        
    r = 2 # num of decimals       
    df_tst_filtered = df_tst_filtered.append(grp.loc[(grp['x'] >= grp['x'].min()) &\
                                               (grp['y'] >= grp['y'].min()) &\
                                               (grp['x'] <= grp['x'].max()) &\
                                               (grp['y'] <= grp['y'].max()) &\
                                               (grp['x'] >= fltr_manual[0][0]) &\
                                               (grp['x'] <= fltr_manual[0][1]) &\
                                               (grp['y'] >= fltr_manual[1][0]) &\
                                               (grp['y'] <= fltr_manual[1][1])])

    
# Single tiles 
# --------------------------------------
# Select Filepath Prediction Dataset
# --------------------------------------
fps_OSTEM = list(Path('/home/rlane/FMML_DATA/20210224_FAST-EM_prototype').glob('*/*_*_*.png'))
fps_BSE = list(Path('/home/rlane/FMML_DATA/20200618_RL012/2us/lil_EM_2us_montaged/').glob('*/*_*_*.png'))
fps_FM = list(Path('/home/rlane/FMML_DATA/20200618_RL012/2us/hoechst_correlated/').glob('*/*_*_*.png'))

df_BSE_FM = clemnet.get_DataFrame(fps_BSE, fps_FM)
fps_BSE = df_BSE_FM.source
fps_FM = df_BSE_FM.target

#521 #628 #1827#1916
fp_OSTEM = fps_OSTEM[1916]
num = 999
fp_BSE = fps_BSE[num]
fp_FM = fps_FM[num]
# 1023

img_OSTEM = img_as_float(io.imread(fp_OSTEM))
img_BSE = img_as_float(io.imread(fp_BSE))
img_FM = img_as_float(io.imread(fp_FM))

# Image PIL enhancements: Sharpness, Contrast, Resizing
# ------------------------------------------------
if adjust_PIL:
    im = Image.open(fp_OSTEM)
    if adjust_invert:
        im = ImageOps.invert(im)
    im.thumbnail((200,200),Image.ANTIALIAS)
    if adjust_sharpness:
        im = ImageEnhance.Sharpness(im)
        im = im.enhance(adjust_sharpness_level)
    if adjust_contrast:
        im = ImageEnhance.Contrast(im)
        im = im.enhance(adjust_contrast_level)
    # im = ImageEnhance.Brightness(im)
    # im = im.enhance(0.0)
    img_OSTEM = np.asarray(im)/255
    # im = Image.fromarray(np.uint8(im))

# # Mean & Std Adjustments + Clipping
# # ------------------------------------------------

# ----------------------------------
# ----------------------------------
mean_tgt_2 = img_OSTEM.mean()
mean_training_2 = img_BSE.mean()
std_tgt_2 = img_OSTEM.std()
std_training_2 = img_BSE.std()
# ----------------------------------
# ----------------------------------
fps_OSTEM = list(Path('/home/rlane/FMML_DATA/20211206_RP_OTO/lil_EM_montaged').glob('*/*_*_*.png'))
fltr_manual_original_mask_size = [[16, 23], [8, 15]]                # zoom_level = 0

In [10]:
timestamp = datetime.now().strftime("%Y%m%d")
filename = f"quant_analysis_{timestamp}.csv"

# Make new df for .csv file
df = pd.DataFrame()

for weights in weights_list:
    print(weights)
    if with_callbacks:
        print('Clipping:            ', adjust_clipping)
        print()
    
    for threshold in tqdm(threshold_value_array):
        if with_callbacks:
            print('Threshold:            ', threshold)
            print()

        for sharpness_level in tqdm(adjust_sharpness_level_array):
            if with_callbacks:
                print('Sharpness:           ', adjust_sharpness,'(level:',sharpness_level,')')
            
            for contrast_level in tqdm(adjust_contrast_level_array):
                if with_callbacks:
                    print('Contrast:            ', adjust_contrast,'(level:',contrast_level,')')
            
                # Load weights
                fp_weights = Path('./weights/'+weights)
                model.load_weights(fp_weights.as_posix())

                # Make figure
                %matplotlib widget

                for z, df_sec in df_tst_filtered.groupby('z'):

                    # Create dataset for section
                    fps_src = df_sec['source']

                    ds_section = clemnet.create_dataset(fps_src, shuffle=False, buffer_size=None,
                                                        repeat=False, shape_src=shape_src, shape_tgt=shape_tgt, augmentations={'invert': adjust_invert},
                                                        pad=pad, padding=padding, batch=True, batch_size=1,
                                                        prefetch=True, n_cores=n_cores)
                    len_ds = tf.data.experimental.cardinality(ds_section).numpy()

                    # Create empty arrays for full section images
                    batch_EM = next(iter(ds_section))
                    shape = ((df_sec['y'].max()-df_sec['y'].min() + 1) * batch_EM.shape[1],
                             (df_sec['x'].max()-df_sec['x'].min() + 1) * batch_EM.shape[2])

                    section_EM = np.empty(shape)
                    section_PR = np.empty(shape)
                    section_mask = np.empty(shape)
                    section_PR_thres_1 = np.empty(shape)
                    section_PR_thres_2 = np.empty(shape)

                    # Loop through individual EM and FM tiles
                    for (i, row), (batch_EM) in zip(df_sec.iterrows(), ds_section):
       
                        file_name_tile = fps_src[i].rsplit('/', 1)[-1]
                        if active_mask == 'FASTEM-0':
                            fps_src_mask = "/home/mabels/CLEMnet/data/FASTEM/masks/"+ str(z) +"/" + file_name_tile
                            img_mask = Image.open(fps_src_mask)
                            img_mask = np.asarray(img_mask)/255
                            img_mask = img_mask[:,:,0]*-1+1

                        if active_mask == 'OSTEM-1':
                            # ---------------------------------------------------------------
                            fps_src_mask = "/home/mabels/CLEMnet/data/OSTEM/masks/0/mask/mask OSTEM.png"
                            img_mask = Image.open(fps_src_mask)

                            img_mask = np.asarray(img_mask)/255
                            y = (row['y']-fltr_manual_original_mask_size[1][0])*scale
                            x = (row['x']-fltr_manual_original_mask_size[0][0])*scale
                            w = scale
                            h = scale

                            img_mask = img_mask[y:y+h, x:x+w, 0]
                            img_mask = resize(img_mask, (1024,1024))
                            img_mask = img_mask[:,:]*-1+1

                            # -------------------------------------        
                            img_tgt = batch_EM[0,:,:,0].numpy()
                            # -------------------------------------


                        img_tgt = batch_EM[0,:,:,0].numpy()
                        img_BSE = img_as_float(io.imread(fps_BSE[i]))

                        # Image PIL enhancements: Sharpness, Contrast, Resizing
                        # ------------------------------------------------
                        if adjust_PIL:
                            fp_tgt = fps_OSTEM[i]
                            im = Image.open(fp_tgt)

                            if adjust_invert:
                                im = ImageOps.invert(im)
                            im.thumbnail((200,200),Image.ANTIALIAS)
                            if adjust_sharpness:
                                im = ImageEnhance.Sharpness(im)
                                im = im.enhance(sharpness_level)
                            if adjust_contrast:
                                im = ImageEnhance.Contrast(im)
                                im = im.enhance(contrast_level)
                            # im = ImageEnhance.Brightness(im)
                            # im = im.enhance(0.0)
                            img_tgt = np.asarray(im)/255
                            img_tgt = resize(img_tgt, (1024,1024))

                        # Mean & Std Adjustments + Clipping
                        # ------------------------------------------------
                        if adjust_mean_std:
                            # ----------------------------------
                            # ----------------------------------
                            mean_tgt_3 = img_tgt.mean()
                            mean_training_3 = img_training.mean()
                            std_tgt_3 = img_tgt.std()
                            std_training_3 = img_BSE.std()

                            mean_tgt = [mean_tgt_1, mean_tgt_2, mean_tgt_3]
                            mean_training = [mean_training_1, mean_training_2, mean_training_3]
                            std_tgt= [std_tgt_1, std_tgt_2, std_tgt_3]
                            std_training = [std_training_1, std_training_2, std_training_3]
                            
                            # select one of 3 possible mean/std sets
                            k=1

                            # ----------------------------------
                            # ----------------------------------
                            img_tgt = mean_std_adjustments(mean_tgt[k], std_tgt[k], mean_training[k], std_training[k], img_tgt)
                        if adjust_clipping:
                            img_tgt = np.clip(img_tgt, 0, 1)

                        # Make prediction
                        img_tgt_predic = img_tgt[np.newaxis,:,:,np.newaxis]
                        prediction = model.predict(img_tgt_predic, verbose=0)

                        pred = prediction[0,:,:,0]
                        pred_threshold_1 = prediction[0,:,:,0]
                        pred_threshold_2 = prediction[0,:,:,0]

                        # Threshold value for mask
                        if threshold_mask:
                            pred_threshold_1 = np.zeros((256, 256)) + pred*(pred > threshold)
                            pred_threshold_2 = np.zeros((256, 256)) + (pred > threshold)

                        # Get indices to populate section images
                        x1 = (df_sec.loc[i, 'x'] - df_sec['x'].min()) * shape_src[1]
                        y1 = (df_sec.loc[i, 'y'] - df_sec['y'].min()) * shape_src[0]
                        x2 = (df_sec.loc[i, 'x'] - df_sec['x'].min() + 1) * shape_src[1]
                        y2 = (df_sec.loc[i, 'y'] - df_sec['y'].min() + 1) * shape_src[0]

                        # Populate section images
                        section_EM[y1:y2, x1:x2] = img_tgt
                        section_PR[y1:y2, x1:x2] = resize(pred, batch_EM.shape[1:3])
                        section_mask[y1:y2, x1:x2] = img_mask
                        section_mask_overlay = (section_mask)*section_PR

                        section_PR_thres_1[y1:y2, x1:x2] = resize(pred_threshold_1, batch_EM.shape[1:3])
                        section_mask_overlay_thres_1 = (section_mask)*section_PR_thres_1

                        section_PR_thres_2[y1:y2, x1:x2] = resize(pred_threshold_2, batch_EM.shape[1:3])
                        section_mask_overlay_thres_2 = (section_mask)*section_PR_thres_2

                    # Colorize and overlay section images
                    section_PR_ = clemnet.colorize(section_PR/section_PR.max(), T=clemnet.T_HOECHST)
                    section_EM_ = color.gray2rgba(section_EM, alpha=True)
                    section_PRED = exposure.rescale_intensity(section_EM_ + section_PR_, in_range=(0, 1))

                    section_PR_thres_1_ = clemnet.colorize(section_PR_thres_1/section_PR_thres_1.max(), T=clemnet.T_HOECHST)
                    section_PRED_thres_1 = exposure.rescale_intensity(section_EM_ + section_PR_thres_1_, in_range=(0, 1))
                    section_PR_thres_2_ = clemnet.colorize(section_PR_thres_2/section_PR_thres_2.max(), T=clemnet.T_HOECHST)
                    section_PRED_thres_2 = exposure.rescale_intensity(section_EM_ + section_PR_thres_2_, in_range=(0, 1)) 

                    # Plot
                    if preview_prediction:
                        output_shape = [d//3 for d in section_EM.shape[:2]] 
                        fig, axmap = plt.subplots(nrows = 3, ncols=4, figsize=(18, 16))
                        fig.subplots_adjust(left=0.01, bottom=0.01, right=None, top=0.999, wspace=0.1, hspace=0.1)
                    #     im1 = axmap[0, 0].imshow(resize(section_EM, output_shape), cmap='Greys_r')
                        im2 = axmap[0, 0].imshow(resize(section_PRED, output_shape))
                        im3 = axmap[0, 1].imshow(resize(section_PR, output_shape), cmap='magma_r')
                        im4 = axmap[0, 2].imshow(resize(section_mask, output_shape), cmap='Greys')
                        im5 = axmap[0, 3].imshow(resize(section_mask_overlay, output_shape), cmap='Greys')

                    #     im1 = axmap[1, 0].imshow(resize(section_EM, output_shape), cmap='Greys_r')
                        im2 = axmap[1, 0].imshow(resize(section_PRED_thres_1, output_shape))
                        im3 = axmap[1, 1].imshow(resize(section_PR_thres_1, output_shape), cmap='magma_r')
                        im4 = axmap[1, 2].imshow(resize(section_mask, output_shape), cmap='Greys')
                        im5 = axmap[1, 3].imshow(resize(section_mask_overlay_thres_1, output_shape), cmap='Greys')

                    #     im1 = axmap[2, 0].imshow(resize(section_EM, output_shape), cmap='Greys_r')
                        im2 = axmap[2, 0].imshow(resize(section_PRED_thres_2, output_shape))
                        im3 = axmap[2, 1].imshow(resize(section_PR_thres_2, output_shape), cmap='magma_r')
                        im4 = axmap[2, 2].imshow(resize(section_mask, output_shape), cmap='Greys')
                        im5 = axmap[2, 3].imshow(resize(section_mask_overlay_thres_2, output_shape), cmap='Greys')

                a4 = sum(sum(section_mask_overlay))
                a3 = sum(sum(section_mask))
                a2 = sum(sum(section_PR))
                alpha_raw = a4/a2 if a2 != 0 else 0
                beta_raw = a4/a3 if a3 != 0 else 0

                b4 = sum(sum(section_mask_overlay_thres_1))
                b3 = sum(sum(section_mask))
                b2 = sum(sum(section_PR_thres_1))
                alpha_thres = b4/b2 if b2 != 0 else 0
                beta_thres = b4/b3 if b3 != 0 else 0

                c4 = sum(sum(section_mask_overlay_thres_2))
                c3 = sum(sum(section_mask))
                c2 = sum(sum(section_PR_thres_2))
                alpha_binary = c4/c2 if c2 != 0 else 0
                beta_binary = c4/c3 if c3 != 0 else 0

                dictionary = {'data': [fp_src],
                        'type': ['prediction'],
                        'zoom_level': [zoom_level],
                        'weights': weights,
                        'mask': [fps_src_mask],

                        'inverted': [adjust_invert],
                        'sharpness': [adjust_sharpness],
                        'sharpness_level': [sharpness_level],
                        'contrast': [adjust_contrast],
                        'contrast_level': [contrast_level],
                        'mean_&_std': [adjust_mean_std],
                        'mean_tgt': [mean_tgt[k]],
                        'mean_training': [mean_training[k]],
                        'std_tgt': [std_tgt[k]],
                        'std_BSE': [std_training[k]],
                        'clipping': [adjust_clipping],
                        'contrast': [adjust_contrast],
                        'threshold': [threshold],

                        'threshold_predic_alpha': [alpha_thres],
                        'threshold_predic_beta': [beta_thres],
                        'threshold_binary_predic_alpha': [alpha_binary],
                        'threshold_binary_predic_beta': [beta_binary], 
                        'product_alpha_beta': (w_alpha*alpha_thres+w_beta*beta_binary), 
                        'w_alpha': w_alpha,
                        'w_beta': w_beta      
                       }

                if save_file:
                    df2 = pd.DataFrame(dictionary)
                    df = df.append(df2)
                    df.to_csv(filename, index=False)
                    data = pd.read_csv(filename)
                    df = pd.DataFrame(data)

                if with_callbacks:
                    print('Inverted:            ', adjust_invert)

                    print('Mean & Std:          ', adjust_mean_std)
                    print()

                    print('Raw pred      alpha:',round(alpha_raw,2),'%')
                    print('              beta: ',round(beta_raw,2),'%')
                    print()


                    print('Thres         alpha:',round(alpha_thres,2),'%')
                    print('              beta: ',round(beta_thres,2),'%')
                    print()
                    print('Thres Binary  alpha:',round(alpha_binary,2),'%')
                    print('              beta: ',round(beta_binary,2),'%')
print('Finished!')


RL012_1024x1024_hoechst_zoomlevel_1_gaussian_blur_1_20220107_151123.h5


  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

2022-03-06 10:00:32.706131: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2022-03-06 10:00:33.787091: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10


...

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

